<a href="https://colab.research.google.com/github/Vakhranev/Pushkina/blob/main/Krippendorff_Alpha_and_so_on.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gspread oauth2client pandas krippendorff

In [37]:
import gspread
import pandas as pd
import numpy as np
from oauth2client.service_account import ServiceAccountCredentials

# Настройка доступа к Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
client = gspread.authorize(creds)

# Список ссылок на таблицы в Google Sheets
spreadsheet_urls = [
    "1IhEkBe0OEHsogaLBd01T-16TTJkkSsSMfoA1XOYX2Uo",
    "1tzai5bypqxsv6GdF5q26K7YFU4vchd51MqFbRYCiMc0",
    "1lUBmSRQCOhl41elAzYeiQWsru1nlX_XqM5sOuwifbrc"
]

# Загрузка данных из Google Sheets
def get_data_from_sheet(sheet_url):
    sheet = client.open_by_key(sheet_url)
    data_frames = {}

    for sheet_name in ["Существительные", "Прилагательные", "Глаголы", "Наречия", "Служебные части речи"]:
        worksheet = sheet.worksheet(sheet_name)

        # Получаем все строки данных
        rows = worksheet.get_all_values()

        if not rows:
            continue

        # Преобразуем в DataFrame
        headers = rows[0]
        data = rows[1:]

        # Создаем DataFrame только с колонками "Слово" и "Термин"
        df = pd.DataFrame(data, columns=headers)
        df = df[['Слово', 'Термин']]
        df = df.drop_duplicates()  # Удаление дубликатов
        df = df.reset_index(drop=True)  # Сброс индексов

        # Добавляем столбец с идентификатором эксперта
        df['Эксперт'] = sheet_url

        # Убедимся, что имена столбцов одинаковы
        df.columns = [col.strip() for col in df.columns]  # Удаление пробелов в названиях столбцов

        data_frames[sheet_name] = df

    return pd.concat(data_frames.values(), ignore_index=True)

# Загрузка данных из файлов Excel
def get_data_from_excel(file_path):
    excel_data = pd.read_excel(file_path, sheet_name=None)
    data_frames = {}

    for sheet_name, df in excel_data.items():
        if sheet_name in ["Существительные", "Прилагательные", "Глаголы", "Наречия", "Служебные части речи"]:
            # Оставляем только столбцы "Слово" и "Термин"
            df = df[['Слово', 'Термин']]
            df = df.drop_duplicates()
            df = df.reset_index(drop=True)

            # Добавляем столбец с идентификатором файла
            df['Эксперт'] = file_path

            # Убедимся, что имена столбцов одинаковы
            df.columns = [col.strip() for col in df.columns]

            data_frames[sheet_name] = df

    return pd.concat(data_frames.values(), ignore_index=True)

# Объединение данных из всех Google Sheets и файлов Excel
google_sheets_data = pd.concat([get_data_from_sheet(url) for url in spreadsheet_urls], ignore_index=True)

# Загрузка данных из Excel-файлов
excel_data_1 = get_data_from_excel("СЛОВАРЬ.xlsx")
excel_data_2 = get_data_from_excel("Словник_Универсальный_словарь_Нечаева_.xlsx")

# Объединяем все данные
all_data = pd.concat([google_sheets_data, excel_data_1, excel_data_2], ignore_index=True)

# Замена пропущенных значений в столбце 'Термин' на 'Не термин'
all_data['Термин'] = all_data['Термин'].fillna('Не термин')

# Преобразование в бинарные значения: 1 для "Термин", 0 для "Не термин"
all_data['Термин'] = all_data['Термин'].apply(lambda x: 1 if str(x).strip().lower() == 'термин' else 0)

# Удаление дублирующихся строк
all_data = all_data.drop_duplicates()

# Пример обработки данных: сводная таблица с оценками терминов по экспертам
word_data = all_data[['Слово', 'Термин', 'Эксперт']].pivot_table(
    index='Слово',
    columns='Эксперт',
    values='Термин',
    aggfunc='max',  # Используем max для объединения оценок, если несколько
    fill_value=0
)

print(word_data)

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Эксперт             1IhEkBe0OEHsogaLBd01T-16TTJkkSsSMfoA1XOYX2Uo  \
Слово                                                              
                                                               0   
Вселенная                                                      1   
абзац                                                          1   
абрикос                                                        0   
абстрактный                                                    0   
...                                                          ...   
ящик                                                           0   
ёжиться, съёжиться                                             0   
ёлочный                                                        0   
ёрш                                                            0   
ёршик                                                          0   

Эксперт             1lUBmSRQCOhl41elAzYeiQWsru1nlX_XqM5sOuwifbrc  \
Слово                                          

In [38]:
# Добавьте функции для расчета коэффициентов
def percentage_agreement(data):
    data = np.array(data)
    total_pairs = 0
    total_agreement = 0
    for i in range(data.shape[0]):
        for j in range(i + 1, data.shape[0]):
            total_pairs += 1
            if np.array_equal(data[i], data[j]):
                total_agreement += 1
    return total_agreement / total_pairs if total_pairs > 0 else 0

def alpha_krippendorff(data):
    """
    Альфа Криппендорфа учитывает согласие между экспертами и рассчитывается на основе
    наблюдаемого несогласия и ожидаемого несогласия.
    """
    data = np.array(data)
    n, k = data.shape

    # Наблюдаемое несогласие
    observed_disagreement = np.sum(np.var(data, axis=1)) / n

    # Ожидаемое несогласие
    category_counts = np.sum(data, axis=0)
    expected_disagreement = np.sum((category_counts / n) * (1 - category_counts / n))

    # Альфа Криппендорфа
    return 1 - (observed_disagreement / expected_disagreement) if expected_disagreement != 0 else 0

def brennan_prediger(data):
    """
    Коэффициент Бреннана-Предига — упрощенная мера согласия, основанная на доле согласных экспертов.
    """
    data = np.array(data)
    n, k = data.shape

    # Средняя доля согласия по категориям
    p_mean = np.mean(np.sum(data, axis=0) / n)

    # Коэффициент Бреннана-Предига
    return (2 * p_mean - 1)

# Расчет коэффициентов
percent_agreement = percentage_agreement(word_data.values)
alpha_krippendorff_value = alpha_krippendorff(word_data.values)
brennan_prediger_value = brennan_prediger(word_data.values)

print(f"Процент согласия: {percent_agreement:.4f}")
print(f"Альфа Криппендорфа: {alpha_krippendorff_value:.4f}")
print(f"Коэффициент Бреннана-Предига: {brennan_prediger_value:.4f}")

Процент согласия: 0.6733
Альфа Криппендорфа: 0.8934
Коэффициент Бреннана-Предига: -0.8681


In [39]:
# Проверка данных на наличие пропущенных значений
missing_values = all_data.isnull().sum()
print("Пропущенные значения в каждом столбце:")
print(missing_values)

# Проверка на дублирующиеся строки
duplicates = all_data.duplicated().sum()
print(f"Количество дублирующихся строк: {duplicates}")

# Проверка уникальных значений в столбце 'Термин'
unique_values = all_data['Термин'].unique()
print("Уникальные значения в столбце 'Термин':")
print(unique_values)

# Поиск слов, по которым эксперты дали противоположные оценки
# Например, если одно и то же слово оценено как 1 (Термин) и 0 (Не термин) разными экспертами
pivot_table = all_data[['Слово', 'Термин', 'Эксперт']].pivot_table(
    index='Слово',
    columns='Эксперт',
    values='Термин',
    aggfunc='max',
    fill_value=0
)

# Выявление строк, где оценки экспертов различаются
disagreement = pivot_table.apply(lambda row: len(set(row)) > 1, axis=1)
disagreement_words = pivot_table[disagreement]
print("Слова, по которым эксперты дали противоположные оценки:")
print(disagreement_words)

# Статистика по количеству терминов и не-терминов для каждого эксперта
expert_stats = all_data.groupby('Эксперт')['Термин'].value_counts().unstack()
print("Статистика по количеству терминов и не-терминов для каждого эксперта:")
print(expert_stats)

Пропущенные значения в каждом столбце:
Слово      0
Термин     0
Эксперт    0
dtype: int64
Количество дублирующихся строк: 0
Уникальные значения в столбце 'Термин':
[1 0]
Слова, по которым эксперты дали противоположные оценки:
Эксперт        1IhEkBe0OEHsogaLBd01T-16TTJkkSsSMfoA1XOYX2Uo  \
Слово                                                         
абстрактный                                               0   
автор                                                     1   
агроном                                                   1   
адаптировать                                              0   
администрация                                             0   
...                                                     ...   
ядовитый                                                  0   
ядро                                                      0   
язык                                                      1   
языковой                                                  0   
янтарный         

In [43]:
import pandas as pd

# Функция для загрузки данных из Google Sheets
def get_data_from_sheet(sheet_url):
    sheet = client.open_by_key(sheet_url)
    data_frames = {}

    for sheet_name in ["Существительные", "Прилагательные", "Глаголы", "Наречия", "Служебные части речи"]:
        worksheet = sheet.worksheet(sheet_name)

        rows = worksheet.get_all_values()
        if not rows:
            continue

        headers = rows[0]
        data = rows[1:]

        # Преобразуем в DataFrame и оставляем нужные колонки
        df = pd.DataFrame(data, columns=headers)
        df = df[['Слово', 'Термин', 'Степень сложности']]  # Добавляем 'Степень сложности'
        df = df.drop_duplicates()
        df = df.reset_index(drop=True)

        df['Эксперт'] = sheet_url

        df.columns = [col.strip() for col in df.columns]
        data_frames[sheet_name] = df

    return pd.concat(data_frames.values(), ignore_index=True)

# Функция для загрузки данных из Excel
def get_data_from_excel(file_path):
    excel_data = pd.read_excel(file_path, sheet_name=None)
    data_frames = {}

    for sheet_name, df in excel_data.items():
        if sheet_name in ["Существительные", "Прилагательные", "Глаголы", "Наречия", "Служебные части речи"]:
            df = df[['Слово', 'Термин', 'Степень сложности']]  # Добавляем 'Степень сложности'
            df = df.drop_duplicates()
            df = df.reset_index(drop=True)

            df['Эксперт'] = file_path
            df.columns = [col.strip() for col in df.columns]

            data_frames[sheet_name] = df

    return pd.concat(data_frames.values(), ignore_index=True)

# Загрузка и объединение всех данных
google_sheets_data = pd.concat([get_data_from_sheet(url) for url in spreadsheet_urls], ignore_index=True)
excel_data_1 = get_data_from_excel("СЛОВАРЬ.xlsx")
excel_data_2 = get_data_from_excel("Словник_Универсальный_словарь_Нечаева_.xlsx")
all_data = pd.concat([google_sheets_data, excel_data_1, excel_data_2], ignore_index=True)

# Обработка термина
all_data['Термин'] = all_data['Термин'].fillna('Не термин')
all_data['Термин'] = all_data['Термин'].apply(lambda x: 1 if str(x).strip().lower() == 'термин' else 0)

# Удаляем дубли слов от одного эксперта
all_data = all_data.drop_duplicates(subset=['Слово', 'Эксперт'])

# Сводка для терминов
terms_summary = all_data.groupby('Слово')['Термин'].sum().reset_index()
terms_summary = terms_summary[terms_summary['Термин'] > 0]  # Исключаем слова, которые не были отмечены как термины
terms_summary = terms_summary.sort_values(by='Термин', ascending=False)

# Сохраняем список терминов в файл
terms_summary.to_csv('Список_терминов.csv', index=False, encoding='utf-8')
print("Список терминов:")
print(terms_summary)

# Обработка сложных слов
all_data['Сложное слово'] = all_data['Степень сложности'].apply(lambda x: 1 if str(x).strip().lower() == 'сложное слово' else 0)

# Сводка для сложных слов
complex_words_summary = all_data.groupby('Слово')['Сложное слово'].sum().reset_index()
complex_words_summary = complex_words_summary[complex_words_summary['Сложное слово'] > 0]  # Исключаем слова, которые не были отмечены как сложные
complex_words_summary = complex_words_summary.sort_values(by='Сложное слово', ascending=False)

# Сохраняем список сложных слов в файл
complex_words_summary.to_csv('Список_сложных_слов.csv', index=False, encoding='utf-8')
print("Список сложных слов:")
print(complex_words_summary)

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Список терминов:
                 Слово  Термин
1            Вселенная       5
5523           потешка       5
4411        орфограмма       5
4413   орфографический       5
4414        орфография       5
...                ...     ...
4319       октябрьский       1
890         выборочный       1
4337  омывать, (омыть)       1
4351          описание       1
8811          янтарный       1

[1609 rows x 2 columns]
Список сложных слов:
                     Слово  Сложное слово
7694              террикон              5
605            взметнуться              5
8434            цветоножка              5
7967                   уда              5
7966                    уд              5
...                    ...            ...
6271            размещение              1
3042                ландыш              1
6273           размножение              1
6274    размывать, размыть              1
4350  опираться, опереться              1

[6191 rows x 2 columns]


<ipython-input-43-049668dc0aff>:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data['Сложное слово'] = all_data['Степень сложности'].apply(lambda x: 1 if str(x).strip().lower() == 'сложное слово' else 0)
